In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
!pip install scikit-learn gensim yellowbrick

In [86]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [82]:
# Load Courses details Data
review_data = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/coursera_reviews_clean_data.csv')

In [83]:
pos_neg_review_data = review_data[review_data["sentiments"] != "Neutral"]

In [84]:
len(pos_neg_review_data)

162990

In [85]:
def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if len(token) > 2]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

pos_neg_review_data["preprocessed_text"] = pos_neg_review_data["reviews"].progress_apply(preprocess)

AttributeError: ignored

In [75]:
# Custom Stop words
stop_words = set(stopwords.words('english'))
# Define a list of negation words and their contractions
negation_words = ["not", "n't", "no", "never", "none", "neither", "nowhere", "nothing",
                  "don't", "doesn't", "didn't", "isn't", "aren't", "wasn't", "weren't",
                  "haven't", "hasn't", "hadn't", "won't", "wouldn't", "can't", "couldn't",
                  "shouldn't"]
    
# Remove all the negation words and their contractions from stop words
custom_stop_words = list({word for word in stop_words if word.lower() not in negation_words})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
tf_vectorizer = CountVectorizer(max_df=0.95,
                                min_df=2,
                                stop_words=custom_stop_words,
                                 ngram_range=(3, 3))
neg_data = pos_neg_review_data[pos_neg_review_data["sentiments"]=="Negative"]["reviews"]
tf_neg = tf_vectorizer.fit_transform(neg_data)

In [67]:
print(tf_vectorizer.get_feature_names_out())

['10 15 minutes' '10 ibm data' '10 min video' ... 'years ago fixed'
 'youtube video course' 'youtube videos subject']


In [77]:
num_topics = 5

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
lda.fit(tf_neg)

LatentDirichletAllocation(n_components=5, random_state=0)

In [78]:
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])

In [79]:
num_top_words = 20

def print_top_words(topic_word_distributions, num_top_words, vectorizer):
    vocab = vectorizer.get_feature_names_out()
    num_topics = len(topic_word_distributions)
    print('Displaying the top %d words per topic and their probabilities within the topic...' % num_top_words)
    print()

    for topic_idx in range(num_topics):
        print('[Topic ', topic_idx, ']', sep='')
        sort_indices = np.argsort(-topic_word_distributions[topic_idx])
        for rank in range(num_top_words):
            word_idx = sort_indices[rank]
            print(vocab[word_idx], ':',
                  topic_word_distributions[topic_idx, word_idx])
        print()

print_top_words(topic_word_distributions, num_top_words, tf_vectorizer)

Displaying the top 20 words per topic and their probabilities within the topic...

[Topic 0]
little bit difficult : 0.007259668055974179
ca not understand : 0.004218034134837845
not good course : 0.003648929203813411
not feel like : 0.0027045337426764423
not take course : 0.0027040122412710357
course learned lot : 0.002703750509134402
could not get : 0.002509583998000809
course extremely informative : 0.0025083429896419727
quite difficult understand : 0.0023210552506367416
first two courses : 0.0023185336264319625
learned lot things : 0.0023182550484064425
course helped understand : 0.002318192879050425
not well explained : 0.002284001532995589
not learn anything : 0.002200665268421665
could not complete : 0.00213016526663926
not explained well : 0.0021281380314200106
course little bit : 0.0018734233682426173
well put together : 0.0017449909011024401
must take course : 0.0015584406102203139
little bit complicated : 0.0015583840301605168

[Topic 1]
hours completecourse approximately : 0

## BERT Topic Modeling

In [ ]:
!pip install bertopic

In [21]:
#import dependencies
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import numpy as np

In [25]:
# Load Courses details Data
Coursera_courses = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/Clean Data/Coursera_courses.csv')

In [ ]:
# Load Courses details Data
review_data = pd.read_csv('/content/drive/MyDrive/CMU SUBJECTS/DATA/temp_coursera_reviews_clean_data.csv')
review_data.head()

In [27]:
final_data = pd.merge(Coursera_courses,review_data,how="inner",on='course_id')

In [34]:
neg_review_data = final_data[final_data["sentiments"] =="Negative"]

In [ ]:
neg_review_data.head()

In [48]:
reviews = neg_review_data["reviews"].reset_index(drop=True)
 

In [49]:
#instantiate BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

#generate the topics
topics, probs = topic_model.fit_transform(reviews)

Batches:   0%|          | 0/473 [00:00<?, ?it/s]

2023-04-15 13:08:00,374 - BERTopic - Transformed documents to Embeddings
2023-04-15 13:08:48,480 - BERTopic - Reduced dimensionality
2023-04-15 13:09:36,868 - BERTopic - Clustered reduced embeddings


In [50]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4966,-1_the_to_and_course
1,0,623,0_quiz_quizzes_questions_answer
2,1,305,1_python_programming_for_you
3,2,275,2_labs_lab_broken_instructions
4,3,270,3_programming_coding_code_for
...,...,...,...
192,191,11,191_sould_difficult_omg_head
193,192,11,192_content_unusual_drags_somewhat
194,193,10,193_courses_clear_trading_hearing
195,194,10,194_ended_indifferent_professors_killing


In [51]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
  diversity=0.5,                      # Diversify topic words
  calculate_probabilities=True,        
  verbose=True
)

topics, probs = topic_model.fit_transform(reviews)
topic_model.get_topic_info()

TypeError: ignored